In [1]:
# Import the required libraries

import os
import zipfile
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile, rmtree, move
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.image import imread
from functools import partial
from datetime import datetime
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
Source_Benign = len(os.listdir('/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/source/benign'))
Source_Melignant = len(os.listdir('/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/source/melignant'))
Source_Total = Source_Benign + Source_Melignant
Source_Melignant_Benign_ratio =  Source_Melignant/Source_Benign

Train_Benign = len(os.listdir('/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/data/train/benign'))
Train_Melignant = len(os.listdir('/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/data/train/melignant'))
Train_Total = Train_Benign + Train_Melignant
Train_Melignant_Benign_ratio =  Train_Melignant/Train_Benign

Validation_Benign = len(os.listdir('/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/data/validation/benign'))
Validation_Melignant = len(os.listdir('/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/data/validation/melignant'))
Validation_Total = Validation_Benign + Validation_Melignant
Validation_Melignant_Benign_ratio =  Validation_Melignant/Validation_Benign

In [3]:
print('Source Benign:', Source_Benign)
print('Source Melignant:', Source_Melignant)
print('Source Total:', Source_Total)
print('Source Melignant/Benign:',Source_Melignant_Benign_ratio)
print('\nTrain Benign:', Train_Benign)
print('Train Melignant:', Train_Melignant)
print('Train Total:', Train_Total)
print('Train Melignant/Benign:', Train_Melignant_Benign_ratio)
print('\nValidation Benign:', Validation_Benign)
print('Validation Melignant:', Validation_Melignant)
print('Validation Total:', Validation_Total)
print('Validation Melignant/Benign:',Validation_Melignant_Benign_ratio)

Source Benign: 32542
Source Melignant: 584
Source Total: 33126
Source Melignant/Benign: 0.017946038965029807

Train Benign: 26033
Train Melignant: 467
Train Total: 26500
Train Melignant/Benign: 0.01793877002266354

Validation Benign: 6509
Validation Melignant: 117
Validation Total: 6626
Validation Melignant/Benign: 0.01797511138423721


In [4]:
# Indicate the paths to the directories again
source_dir_path = '/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/source'
source_benign_path = os.path.join(source_dir_path, 'benign')
source_melignant_path = os.path.join(source_dir_path, 'melignant')
data_dir_path = '/sfs/lustre/bahamut/scratch/uk7ud/Kaggle Melanoma/data'
train_dir_path = os.path.join(data_dir_path, 'train')
train_benign_dir_path = os.path.join(train_dir_path, 'benign')
train_melignant_dir_path = os.path.join(train_dir_path, 'melignant')
validation_dir_path = os.path.join(data_dir_path, 'validation')
validation_benign_dir_path = os.path.join(validation_dir_path, 'benign')
validation_melignant_dir_path = os.path.join(validation_dir_path, 'melignant')

In [5]:
input_shape = (144, 144, 3)

In [6]:
# Prepare the data using ImageDataGenerator API from keras and also include data augmentation

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.4,
    height_shift_range = 0.4,
    shear_range = 0.2,
    zoom_range = 0.4,
    horizontal_flip = True,
    vertical_flip = True,
    brightness_range=(0.2,0.6),
    fill_mode = 'nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir_path,
    batch_size = 8,
    target_size = input_shape[:2],
    class_mode = 'binary'
) 

validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir_path,
    batch_size = 8,
    target_size = input_shape[:2],
    class_mode = 'binary'
)

Found 26500 images belonging to 2 classes.
Found 6626 images belonging to 2 classes.


In [7]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, padding="SAME")

model = keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=3, input_shape=input_shape),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=128),
    keras.layers.LeakyReLU(alpha=0.2),
    DefaultConv2D(filters=128),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    keras.layers.LeakyReLU(alpha=0.2),
    DefaultConv2D(filters=256),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=1, activation='sigmoid')
])

model.compile(
    optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True),
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC()])

model.summary()

ResourceExhaustedError: OOM when allocating tensor with shape[82944,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Mul] name: dense/kernel/Initializer/random_uniform/mul/

In [ ]:
logs = os.path.join(os.curdir, "my_logs", "run_" + datetime.now().strftime("%Y%m%d_%H%M%S"))
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=logs, histogram_freq=1, profile_batch=10)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_model_tfrecords.h5", save_best_only=True)

In [8]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  4


In [9]:
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)


RuntimeError: Device placement logging must be set at program startup

In [10]:
tf.debugging.set_log_device_placement(True)

# Place tensors on the CPU
with tf.device('/CPU:0'):
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

c = tf.matmul(a, b)
print(c)

RuntimeError: Device placement logging must be set at program startup